In [1]:
import numpy as np
import ray
import matplotlib.pyplot as plt
import time
from utils import *
from procedures import *
plt.rcParams["font.sans-serif"] = "DejaVu Sans"
plt.rcParams["mathtext.default"]  = "it"
plt.rcParams["mathtext.fontset"]  = "dejavusans"

In [2]:
# Number of parallel threads to be used
# Shut down any existing Ray processes
# Initialize Ray with the specified number of parallel threads

num_cpus=96
ray.shutdown()
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

2023-07-01 17:01:52,752	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.192.9.221',
 'raylet_ip_address': '10.192.9.221',
 'redis_address': '10.192.9.221:6379',
 'object_store_address': 'tcp://127.0.0.1:64105',
 'raylet_socket_name': 'tcp://127.0.0.1:65478',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\ray\\session_2023-07-01_17-01-47_715878_21556',
 'metrics_export_port': 64087,
 'node_id': 'a70617b06a3235813bde238bc02687141c6abc6764f2cfa6a354295c'}

In [3]:
from TpMaxSimulator import TpMaxSimulator

In [ ]:
# Rewrite the TpMaxGenerator function of the utils.py
# The __init__ is slightly updated compared to the original version during the review process.
# To produce exactly the same results, we write this version here.

class TpMaxGenerator(object):
    """
    Generator class for throughput maximization (TP) simulations.

    Attributes:
        _njobs (int): Number of jobs.
        _nstages (int): Number of stages.
        _burnin (int): Burn-in parameter.
        slns (ndarray): Array of solution vectors.
        means (ndarray): Array of means.
        best_mean (float): Maximum mean value.
        simulator (function): Simulator function for TP.
    """

    def __init__(self, _njobs,  _nstages, _burnin, slns, means, simulator):
        """
        Initialize the TpMaxGenerator with the provided parameters.

        Args:
            _njobs (int): Number of jobs.
            _nstages (int): Number of stages.
            _burnin (int): Burn-in parameter.
            slns (ndarray): Array of solution vectors.
            means (ndarray): Array of means.
            simulator (function): Simulator function for TP.
        """

        self._njobs = _njobs
        self._nstages = _nstages
        self._burnin = _burnin
        # suffle the orders
        ids = np.arange(len(slns))
        self.slns = slns
        self.means = means
        self.best_mean = np.max(self.means)
        self.simulator = simulator
        
    def get(self, index, n=1):
        """
        Generate TP simulations based on the specified index.

        Args:
            index (int): Index of the solution vector.
            n (int): Number of simulations to generate (default is 1).

        Returns:
            ndarray: Array of TP simulation results.
        """

        r = self.slns[index][:self._nstages]
        b = self.slns[index][self._nstages:]
        if n == 1:
            return TpMax(self._njobs, self._nstages, self._burnin, r, b, self.simulator)
        else:
            results = [TpMax(self._njobs, self._nstages, self._burnin, r, b, self.simulator) for i in range(n)]
            return np.array(results)
        
    def syscount(self):
        return len(self.slns)
    
    def getbest(self):
        return self.best_mean

In [4]:
recipe = [30, 50, 100, 200]
n_recipe = len(recipe)
n_replications = 500
G = 11
_njobs, _nstages, _burnin = 1050, 3, 1000

In [5]:
SH_results = []

In [6]:
problem_types = [(20, 20), (30, 30), (45, 30), (45, 45)]
seed = 20230701
np.random.seed(seed)
rng = np.random.default_rng(seed)
for problem_type in problem_types:
    slns, means = loadTPMax(problem_type[0], problem_type[1])
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        n = recipe[i]
        print("Sequential Halving with n:{}".format(n))
        PCS, PGS = parallel_experiments(rng,  generators, policy=SH, remote_policy=remote_SH,  args={"n":n})
        SH_results.append([PCS,PGS])

problem size:3249
Sequential Halving with n:30
--------New experiments with  k=3249----------------------
Start to simulate... at Sat Jul  1 17:02:06 2023
PCS:0.482, PGS:0.976
Total time used: 87.10181379318237s, simulation ends at Sat Jul  1 17:03:33 2023
Sequential Halving with n:50
--------New experiments with  k=3249----------------------
Start to simulate... at Sat Jul  1 17:03:33 2023
PCS:0.574, PGS:0.994
Total time used: 146.85943341255188s, simulation ends at Sat Jul  1 17:06:00 2023
Sequential Halving with n:100
--------New experiments with  k=3249----------------------
Start to simulate... at Sat Jul  1 17:06:00 2023
PCS:0.632, PGS:1.0
Total time used: 293.76562428474426s, simulation ends at Sat Jul  1 17:10:53 2023
Sequential Halving with n:200
--------New experiments with  k=3249----------------------
Start to simulate... at Sat Jul  1 17:10:54 2023
PCS:0.688, PGS:1.0
Total time used: 589.7500927448273s, simulation ends at Sat Jul  1 17:20:43 2023
problem size:11774
Sequent